In [ ]:
import numpy as np
import pymc as pm
import pytensor.tensor as pt
import arviz as az
import matplotlib.pyplot as plt
from scipy import signal
from collections import deque

In [ ]:
def train_dynamic_ar_model(train_data, d=1, step=1, tau_sec=3):
    """
    Complete corrected version: AR coefficients update every tau seconds (block-wise random walk).
    Uses AR correction principle from first function: v_corrected = v_base + Σ ρ[lag] × (vt_diff - a_lag × dt)
    """
    print("Starting DYNAMIC AR model training...")
    
    vt = train_data['vt']
    s = train_data['s']
    dv = train_data['dv']
    label_v = train_data['label_v']
    id_idx = train_data['id_idx']
    N_veh = train_data['n_vehicles']
    
    dt = 0.5
    D = 5  # IDM parameters
    DELTA = 4
    tau = int(tau_sec / dt)  # block size
    
    coords = {
        "veh_id": np.arange(N_veh),
        "ar_lag": np.arange(d),
        "parameter": np.arange(D)
    }
    
    with pm.Model(coords=coords) as dynamic_ar_model:
        # Hierarchical IDM parameter structure
        chol, _, _ = pm.LKJCholeskyCov('chol', n=D, eta=2.0, 
                                      sd_dist=pm.Exponential.dist(2, shape=D))
        
        # Population mean parameters
        log_mu_vmax = pm.Normal('log_mu_vmax', mu=0, sigma=0.1)
        log_mu_dsafe = pm.Normal('log_mu_dsafe', mu=0, sigma=0.1)
        log_mu_tsafe = pm.Normal('log_mu_tsafe', mu=0, sigma=0.1)
        log_mu_amax = pm.Normal('log_mu_amax', mu=0, sigma=0.1)
        
        # Ratio parameter setup
        log_ratio_mu = pm.Normal('log_ratio_mu', mu=np.log(1.0), sigma=0.1)
        ratio_mu = pm.Deterministic('ratio_mu', pt.exp(log_ratio_mu))
        
        # Individual ratio differences
        log_ratio_raw = pm.Normal('log_ratio_raw', mu=0, sigma=0.1, shape=N_veh)
        ratio_individual = pm.Deterministic('ratio_individual', ratio_mu * pt.exp(log_ratio_raw))
        
        # Individual random effects
        vals_raw_vmax = pm.Normal('vals_raw_vmax', mu=0, sigma=0.1, shape=N_veh)
        vals_raw_dsafe = pm.Normal('vals_raw_dsafe', mu=0, sigma=0.1, shape=N_veh)
        vals_raw_tsafe = pm.Normal('vals_raw_tsafe', mu=0, sigma=0.1, shape=N_veh)
        vals_raw_amax = pm.Normal('vals_raw_amax', mu=0, sigma=0.1, shape=N_veh)
        
        # Combine into matrix form (first 4 parameters)
        vals_raw_first4 = pm.Deterministic('vals_raw_first4', pt.stack([
            vals_raw_vmax, vals_raw_dsafe, vals_raw_tsafe, vals_raw_amax
        ], axis=1))
        
        # Calculate individual parameters for first 4 parameters
        log_mu_first4 = pt.stack([log_mu_vmax, log_mu_dsafe, log_mu_tsafe, log_mu_amax])
        log_parameters_first4 = pm.Deterministic('log_parameters_first4', 
                                               log_mu_first4 + pt.dot(vals_raw_first4, chol[:4, :4].T))
        parameters_first4 = pm.Deterministic('parameters_first4', pt.exp(log_parameters_first4))
        
        # Correct matrix concatenation
        amin_individual = ratio_individual * parameters_first4[:, 3]  # amin = ratio * amax
        
        # Combine all parameters
        parameters = pm.Deterministic('parameters', pt.stack([
            parameters_first4[:, 0],  # vmax
            parameters_first4[:, 1],  # dsafe
            parameters_first4[:, 2],  # tsafe
            parameters_first4[:, 3],  # amax
            amin_individual            # amin
        ], axis=1), dims=('veh_id', 'parameter'))
        
        # Non-hierarchical parameters
        s_a_list = []
        s_v_list = []
        for i in range(N_veh):
            s_a_i = pm.Exponential(f's_a_{i}', lam=2000)
            s_v_i = pm.Exponential(f's_v_{i}', lam=4000)
            s_a_list.append(s_a_i)
            s_v_list.append(s_v_i)
        
        # Dynamic AR hyperparameters
        rho_transition = pm.Deterministic('rho_transition', pt.zeros((N_veh, d)))
        rho_innovation = pm.HalfNormal('rho_innovation', sigma=0.001, dims=("veh_id", "ar_lag"))
        rho_0 = pm.Normal('rho_0', mu=0, sigma=0.4, dims=("veh_id", "ar_lag"))
        rho_dynamic = [None] * N_veh
        rho_block_info = {}  # Store block information for validation
        
        for i in range(N_veh):
            mask = (id_idx == i)
            n_veh_timesteps = int(np.sum(mask))
            if n_veh_timesteps > (d * step) + 5:
                n_blocks = int(np.ceil(n_veh_timesteps / tau))
                
                rho_lag_list = []
                rho_block_values_list = []  # Store block values for each lag
                
                for lag in range(d):
                    mu_scalar = rho_transition[i, lag]
                    sigma_scalar = rho_innovation[i, lag]
                    init_mu = rho_0[i, lag]
                    
                    # block-wise initial value
                    rho_init = pm.Normal(f'rho_init_{i}_lag{lag}', mu=init_mu, sigma=0.1)
                    
                    # block innovation terms
                    if n_blocks > 1:
                        innov_blocks = pm.Normal(
                            f'rho_innov_blocks_{i}_lag{lag}',
                            mu=0.0,
                            sigma=rho_innovation[i, lag], 
                            shape=(n_blocks - 1,)
                        )
                        cumsum_blocks = pm.math.cumsum(innov_blocks)
                        # Concatenate into block AR values
                        rho_block_vals = pm.math.concatenate([pm.math.stack([rho_init]), rho_init + cumsum_blocks])
                    else:
                        rho_block_vals = pm.math.stack([rho_init])
                    
                    rho_block_values_list.append(rho_block_vals)
                    
                    # Expand to time series, constant within each block
                    rho_vals = pt.repeat(rho_block_vals, tau)[:n_veh_timesteps]
                    rho_lag_list.append(rho_vals)
                
                # Store block information for validation
                rho_block_info[i] = {
                    'n_blocks': n_blocks,
                    'block_values': rho_block_values_list,
                    'tau': tau
                }
                
                # Stack into (n_veh_timesteps, d)
                rho_i_stack = pm.math.stack(rho_lag_list, axis=1)
                rho_dynamic[i] = rho_i_stack
            else:
                rho_dynamic[i] = None
                rho_block_info[i] = None
        
        # Observation model
        for i in range(N_veh):
            mask = (id_idx == i)
            if rho_dynamic[i] is not None:
                s_veh = s[mask]
                vt_veh = vt[mask]
                dv_veh = dv[mask]
                label_veh = label_v[mask]
                n_veh = len(vt_veh)
                
                # Use hierarchical IDM parameters (with scaling factors)
                vmax_i = 25 * parameters[i, 0]
                dsafe_i = 2  * parameters[i, 1]
                tsafe_i = 1.6 * parameters[i, 2]
                amax_i = 1.5   * parameters[i, 3]
                amin_i = 1.5  * parameters[i, 4]
                
                sn = dsafe_i + vt_veh * tsafe_i + vt_veh * dv_veh / (2 * pm.math.sqrt(amax_i * amin_i))
                a = amax_i * (1 - (vt_veh / vmax_i) ** DELTA - (sn / s_veh) ** 2)
                
                # Key modification: AR correction principle from first function
                # Base speed prediction (without AR correction)
                mean = vt_veh + a * dt
                
                # Apply AR correction - speed level
                total_ar_correction = pt.zeros_like(mean)
                
                rho_i_dynamic = rho_dynamic[i]
                n = n_veh
                
                for lag in range(d):
                    if n > lag + 1:
                        # Calculate speed difference and acceleration terms
                        vt_diff = vt_veh[lag+1:n] - vt_veh[lag:n-1]
                        a_lag = a[lag:n-1]
                        
                        # AR correction term: rho * (speed difference - acceleration integral)
                        ar_correction = rho_i_dynamic[lag+1:n, lag] * (vt_diff - a_lag * dt)
                        
                        # Apply correction to speed prediction (from d to end)
                        start_idx = max(d, lag+1)
                        if start_idx < n:
                            correction_length = n - start_idx
                            ar_slice = ar_correction[start_idx-lag-1:start_idx-lag-1+correction_length]
                            total_ar_correction = pt.set_subtensor(
                                total_ar_correction[start_idx:n],
                                total_ar_correction[start_idx:n] + ar_slice
                            )
                
                # Apply AR correction for final speed prediction
                mean_corrected = mean + total_ar_correction
                
                total_sigma = pm.math.sqrt((s_a_list[i] * dt) ** 2 + s_v_list[i] ** 2)
                pm.Normal(f'obs_{i}', mu=mean_corrected, sigma=total_sigma, observed=label_veh)
        
        # Sampling
        try:
            trace = pm.sample(draws=800, tune=800, random_seed=42, chains=2, target_accept=0.95, max_treedepth=20, return_inferencedata=True)
            print("Dynamic AR model training completed!")
        except Exception as e:
            print(f"Sampling error: {e}")
            print("Trying more conservative sampling settings...")
            trace = pm.sample(draws=1000, tune=1000, random_seed=42, chains=2, target_accept=0.8, max_treedepth=20, return_inferencedata=True)
            print("Dynamic AR model training completed (using conservative settings)!")
    
    return trace, dynamic_ar_model, rho_block_info

In [ ]:
def calculate_metrics(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    nrmse = rmse / (np.max(y_true) - np.min(y_true))
    
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'nrmse': nrmse
    }

def robust_smooth_acceleration(velocity, dt=0.5, window_size=7, poly_order=2):
    if len(velocity) < window_size:
        acceleration = np.gradient(velocity, dt)
        return acceleration
    
    try:
        acceleration = signal.savitzky_golay(velocity, window_length=window_size, 
                                           polyorder=poly_order, deriv=1, delta=dt)
        
        if len(acceleration) > 10:
            x_fit = np.arange(5) * dt
            y_fit = acceleration[5:10]
            if len(y_fit) >= 2:
                slope, intercept = np.polyfit(x_fit[:len(y_fit)], y_fit, 1)
                for i in range(5):
                    acceleration[i] = intercept + slope * (i * dt)
            
            x_fit = np.arange(5) * dt
            y_fit = acceleration[-10:-5]
            if len(y_fit) >= 2:
                slope, intercept = np.polyfit(x_fit[:len(y_fit)], y_fit, 1)
                for i in range(5):
                    acceleration[-(5-i)] = intercept + slope * ((4-i) * dt)
        
        return acceleration
        
    except:
        acceleration = np.zeros_like(velocity)
        for i in range(1, len(velocity)-1):
            acceleration[i] = (velocity[i+1] - velocity[i-1]) / (2 * dt)
        
        if len(velocity) > 1:
            acceleration[0] = (velocity[1] - velocity[0]) / dt
            acceleration[-1] = (velocity[-1] - velocity[-2]) / dt
        
        window = min(5, len(acceleration))
        acceleration = np.convolve(acceleration, np.ones(window)/window, mode='same')
        
        return acceleration

def calculate_initial_acceleration(velocity, dt=0.5, method='savitzky_golay'):
    if len(velocity) < 3:
        return 0.0
    
    if method == 'savitzky_golay':
        acc_all = robust_smooth_acceleration(velocity, dt)
        return acc_all[0]
    
    elif method == 'robust_fit':
        n_points = min(5, len(velocity))
        t_points = np.arange(n_points) * dt
        v_points = velocity[:n_points]
        
        slope, intercept = np.polyfit(t_points, v_points, 1)
        return slope
    
    elif method == 'physical_constrained':
        if len(velocity) >= 4:
            weights = np.array([0.1, 0.2, 0.3, 0.4])[:len(velocity)]
            weights = weights / np.sum(weights)
            
            t_points = np.arange(len(velocity)) * dt
            A = np.vstack([t_points, np.ones(len(t_points))]).T
            W = np.diag(weights)
            slope, intercept = np.linalg.lstsq(A.T @ W @ A, A.T @ W @ velocity, rcond=None)[0]
            return slope
        else:
            return (velocity[1] - velocity[0]) / dt
    
    else:
        initial_acc = (velocity[1] - velocity[0]) / dt
        return np.clip(initial_acc, -3.0, 3.0)

def improved_robust_acceleration(velocity, dt=0.5, window_size=7, poly_order=2):
    acceleration = robust_smooth_acceleration(velocity, dt, window_size, poly_order)
    
    acceleration = np.clip(acceleration, -3.0, 3.0)
    
    if len(acceleration) > 10:
        acceleration[:3] = np.mean(acceleration[:5])
        acceleration[-3:] = np.mean(acceleration[-5:])
    
    return acceleration

def split_data_for_ar_idm(ar_idm_data, train_ratio=0.7):
    vt = ar_idm_data['vt']
    s = ar_idm_data['s']
    dv = ar_idm_data['dv']
    label_v = ar_idm_data['label_v']
    id_idx = ar_idm_data['id_idx']
    
    unique_vehicles = np.unique(id_idx)
    
    train_data = {
        'vt': np.array([]),
        's': np.array([]),
        'dv': np.array([]),
        'label_v': np.array([]),
        'id_idx': np.array([], dtype=int),
        'n_vehicles': ar_idm_data['n_vehicles'],
        'tracks': {}
    }
    
    val_data = {
        'vt': np.array([]),
        's': np.array([]),
        'dv': np.array([]),
        'label_v': np.array([]),
        'id_idx': np.array([], dtype=int),
        'n_vehicles': ar_idm_data['n_vehicles'],
        'tracks': {}
    }
    
    val_data1 = {
        'vt': np.array([]),
        's': np.array([]),
        'dv': np.array([]),
        'label_v': np.array([]),
        'id_idx': np.array([], dtype=int),
        'n_vehicles': ar_idm_data['n_vehicles'],
        'tracks': {}
    }
    
    for veh_id in unique_vehicles:
        mask = (id_idx == veh_id)
        n_points = np.sum(mask)
        
        if n_points < 20:
            continue
            
        split_point = int(n_points * train_ratio)
        
        train_mask = np.zeros_like(mask, dtype=bool)
        train_indices = np.where(mask)[0][:split_point]
        train_mask[train_indices] = True
        
        val_mask = np.zeros_like(mask, dtype=bool)
        val_indices = np.where(mask)[0][split_point:]
        val_mask[val_indices] = True
        
        val1_mask = np.zeros_like(mask, dtype=bool)
        val1_indices = np.where(mask)[0][split_point:split_point + 100]
        val1_mask[val1_indices] = True
        
        train_data['vt'] = np.concatenate([train_data['vt'], vt[train_mask]])
        train_data['s'] = np.concatenate([train_data['s'], s[train_mask]])
        train_data['dv'] = np.concatenate([train_data['dv'], dv[train_mask]])
        train_data['label_v'] = np.concatenate([train_data['label_v'], label_v[train_mask]])
        train_data['id_idx'] = np.concatenate([train_data['id_idx'], np.full(np.sum(train_mask), veh_id)])
        
        if np.sum(train_mask) > 0:
            train_data['tracks'][veh_id] = {
                'last_vt': vt[train_mask][-1],
                'last_s': s[train_mask][-1],
                'last_dv': dv[train_mask][-1] if len(dv[train_mask]) > 0 else 0.0,
                'last_rho_blocks': {}
            }
        
        val_data['vt'] = np.concatenate([val_data['vt'], vt[val_mask]])
        val_data['s'] = np.concatenate([val_data['s'], s[val_mask]])
        val_data['dv'] = np.concatenate([val_data['dv'], dv[val_mask]])
        val_data['label_v'] = np.concatenate([val_data['label_v'], label_v[val_mask]])
        val_data['id_idx'] = np.concatenate([val_data['id_idx'], np.full(np.sum(val_mask), veh_id)])
        
        if np.sum(val1_mask) > 0:
            val_data1['vt'] = np.concatenate([val_data1['vt'], vt[val1_mask]])
            val_data1['s'] = np.concatenate([val_data1['s'], s[val1_mask]])
            val_data1['dv'] = np.concatenate([val_data1['dv'], dv[val1_mask]])
            val_data1['label_v'] = np.concatenate([val_data1['label_v'], label_v[val1_mask]])
            val_data1['id_idx'] = np.concatenate([val_data1['id_idx'], np.full(np.sum(val1_mask), veh_id)])
    
    print(f"Training set: {len(train_data['vt'])} data points")
    print(f"Full validation set: {len(val_data['vt'])} data points")
    print(f"Val_data1 (first 30s): {len(val_data1['vt'])} data points")
    
    return train_data, val_data, val_data1

In [ ]:
def validate_dynamic_ar_model_comprehensive(trace, model, train_data, val_data, rho_block_info, n_samples=50, d=1, step=1, tau_sec=5):
    print("\nStarting DYNAMIC AR model validation with posterior sampling...")
    
    n_chains = len(trace.posterior.chain)
    n_draws = len(trace.posterior.draw)
    
    sample_indices = []
    for _ in range(n_samples):
        chain_idx = np.random.randint(0, n_chains)
        draw_idx = np.random.randint(0, n_draws)
        sample_indices.append((chain_idx, draw_idx))
    
    vt_val = val_data['vt']
    s_val = val_data['s'] 
    dv_val = val_data['dv']
    label_val = val_data['label_v']
    id_idx_val = val_data['id_idx']
    
    dt = 0.5
    DELTA = 4
    tau = int(tau_sec / dt)
    
    print("Calculating real acceleration using improved robust method...")
    real_acceleration_all = improved_robust_acceleration(vt_val, dt)
    
    all_samples_speed_predictions = []
    all_samples_spacing_predictions = []
    all_samples_acceleration_predictions = []
    all_samples_ar_coefficients = []
    all_samples_vehicle_predictions = {}
    
    print("Extracting numerical block information for all vehicles...")
    numerical_block_info = {}
    
    for veh_id in range(val_data['n_vehicles']):
        train_mask = (train_data['id_idx'] == veh_id)
        n_train_timesteps = int(np.sum(train_mask))
        
        if n_train_timesteps > (d * step) + 5:
            n_train_blocks = int(np.ceil(n_train_timesteps / tau))
            
            block_values_numerical = []
            for lag in range(d):
                try:
                    init_key = f'rho_init_{veh_id}_lag{lag}'
                    if init_key in trace.posterior:
                        init_val = trace.posterior[init_key].mean(dim=("chain", "draw")).item()
                    else:
                        init_val = trace.posterior['rho_0'].mean(dim=("chain", "draw")).sel(veh_id=veh_id, ar_lag=lag).item()
                    
                    innov_key = f'rho_innov_blocks_{veh_id}_lag{lag}'
                    if innov_key in trace.posterior:
                        innov_vals = trace.posterior[innov_key].mean(dim=("chain", "draw")).values
                    else:
                        innov_vals = np.zeros(max(0, n_train_blocks - 1))
                    
                    block_vals = [init_val]
                    for innov in innov_vals:
                        block_vals.append(block_vals[-1] + innov)
                    
                    block_values_numerical.append(np.array(block_vals))
                    
                except Exception as e:
                    default_val = trace.posterior['rho_0'].mean(dim=("chain", "draw")).sel(veh_id=veh_id, ar_lag=lag).item()
                    block_values_numerical.append(np.array([default_val]))
            
            numerical_block_info[veh_id] = {
                'n_blocks': n_train_blocks,
                'block_values': block_values_numerical,
                'tau': tau,
                'n_timesteps': n_train_timesteps
            }
        else:
            numerical_block_info[veh_id] = None
    
    print(f"Extracted numerical block information for {len([x for x in numerical_block_info.values() if x is not None])} vehicles")
    
    for sample_idx, (chain_idx, draw_idx) in enumerate(sample_indices):
        print(f"Processing posterior sample {sample_idx + 1}/{n_samples}...")
        
        individual_params = trace.posterior['parameters'].sel(chain=chain_idx, draw=draw_idx).values
        rho_0 = trace.posterior['rho_0'].sel(chain=chain_idx, draw=draw_idx).values
        rho_innovation = trace.posterior['rho_innovation'].sel(chain=chain_idx, draw=draw_idx).values
        
        rho_innov_blocks_dict = {}
        rho_init_dict = {}
        
        for i in range(val_data['n_vehicles']):
            for lag in range(d):
                try:
                    innov_key = f'rho_innov_blocks_{i}_lag{lag}'
                    if innov_key in trace.posterior:
                        rho_innov_blocks_dict[(i, lag)] = trace.posterior[innov_key].sel(
                            chain=chain_idx, draw=draw_idx).values
                    
                    init_key = f'rho_init_{i}_lag{lag}'
                    if init_key in trace.posterior:
                        rho_init_dict[(i, lag)] = trace.posterior[init_key].sel(
                            chain=chain_idx, draw=draw_idx).values
                except:
                    pass
        
        all_speed_predictions = []
        all_spacing_predictions = []
        all_acceleration_predictions = []
        all_ar_coefficients_veh = []
        sample_vehicle_predictions = {}
        
        unique_vehicles = np.unique(id_idx_val)
        
        for veh_id in unique_vehicles:
            mask = (id_idx_val == veh_id)
            if np.sum(mask) > d + 5:
                vt_veh = vt_val[mask]
                s_veh = s_val[mask]
                dv_veh = dv_val[mask]
                real_accel_veh = real_acceleration_all[mask]
                
                initial_v = vt_veh[0]
                initial_s = s_veh[0]
                initial_dv = dv_veh[0] if len(dv_veh) > 0 else 0.0
                
                if veh_id in train_data['tracks']:
                    train_mask = (train_data['id_idx'] == veh_id)
                    if np.sum(train_mask) >= 5:
                        train_vt = train_data['vt'][train_mask]
                        initial_acc = calculate_initial_acceleration(train_vt[-5:], dt, method='savitzky_golay')
                    else:
                        initial_acc = 0.0
                else:
                    initial_acc = 0.0
                
                vmax = 25 * individual_params[veh_id, 0]
                dsafe = 2 * individual_params[veh_id,1]
                tsafe = 1.6 * individual_params[veh_id, 2]
                amax = 1.5 * individual_params[veh_id, 3]
                amin = 1.5 * individual_params[veh_id, 4]
                
                initial_acc = np.clip(initial_acc, -amin, amax)
                
                warmup_steps = 5
                
                n_val_points = len(vt_veh)
                n_val_blocks = int(np.ceil(n_val_points / tau))
                
                rho_dynamic_val = np.zeros((n_val_points, d))
                
                for lag in range(d):
                    rho_0_val = rho_0[veh_id, lag]
                    
                    if veh_id in numerical_block_info and numerical_block_info[veh_id] is not None:
                        train_block_values = numerical_block_info[veh_id]['block_values'][lag]
                        if len(train_block_values) > 0:
                            last_train_block_value = train_block_values[-1]
                        else:
                            last_train_block_value = rho_0_val
                    else:
                        if (veh_id, lag) in rho_init_dict:
                            last_train_block_value = rho_init_dict[(veh_id, lag)]
                        else:
                            last_train_block_value = rho_0_val
                    
                    if n_val_blocks > 0:
                        if (veh_id, lag) in rho_innov_blocks_dict:
                            innov_available = rho_innov_blocks_dict[(veh_id, lag)]
                            n_innov_needed = n_val_blocks
                            if len(innov_available) >= n_innov_needed:
                                val_innov = innov_available[:n_innov_needed]
                            else:
                                n_additional = n_innov_needed - len(innov_available)
                                additional_innov = np.random.normal(0, rho_innovation[veh_id, lag], n_additional)
                                val_innov = np.concatenate([innov_available, additional_innov])
                        else:
                            val_innov = np.random.normal(0, rho_innovation[veh_id, lag], n_val_blocks)
                        
                        val_block_values = [last_train_block_value]
                        for i in range(n_val_blocks):
                            new_value = val_block_values[-1] + val_innov[i]
                            val_block_values.append(new_value)
                        
                        val_block_values = val_block_values[1:]
                        
                        for block_idx, block_value in enumerate(val_block_values):
                            start_idx = block_idx * tau
                            end_idx = min((block_idx + 1) * tau, n_val_points)
                            rho_dynamic_val[start_idx:end_idx, lag] = block_value
                    else:
                        rho_dynamic_val[:, lag] = last_train_block_value
                
                all_ar_coefficients_veh.append(rho_dynamic_val)
                
                v_sim = [initial_v]
                s_sim = [initial_s]
                a_sim = [initial_acc]
                
                v_history_real = deque([initial_v], maxlen=d+1)
                idm_acc_history = deque([initial_acc], maxlen=d)
                acceleration_errors = deque([], maxlen=d)
                
                n = len(vt_veh)
                
                for i in range(n-1):
                    if i < warmup_steps and i < len(vt_veh) - 1:
                        v_next_real = vt_veh[i + 1] if (i + 1) < len(vt_veh) else vt_veh[i]
                        s_next_real = s_veh[i + 1] if (i + 1) < len(s_veh) else s_veh[i]
                        a_next_real = real_accel_veh[i] if i < len(real_accel_veh) else 0.0
                        
                        s_next_real = max(1.0, min(200.0, s_next_real))
                        a_next_real = np.clip(a_next_real, -amin, amax)
                        
                        v_sim.append(v_next_real)
                        s_sim.append(s_next_real)
                        a_sim.append(a_next_real)
                        
                        v_history_real.append(v_next_real)
                        
                        current_dv = dv_veh[i] if i < len(dv_veh) else 0.0
                        sn = dsafe + v_sim[-2] * tsafe + v_sim[-2] * current_dv / (2 * np.sqrt(amax * amin))
                        current_acc_idm = amax * (1 - (v_sim[-2] / vmax) ** DELTA - (sn / s_sim[-2]) ** 2)
                        current_acc_idm = np.clip(current_acc_idm, -amin, amax)
                        idm_acc_history.append(current_acc_idm)
                        
                        if i > 0:
                            a_real_prev = real_accel_veh[i-1] if (i-1) < len(real_accel_veh) else 0.0
                            prev_dv = dv_veh[i-1] if (i-1) < len(dv_veh) else 0.0
                            prev_sn = dsafe + v_sim[-3] * tsafe + v_sim[-3] * prev_dv / (2 * np.sqrt(amax * amin))
                            a_idm_prev = amax * (1 - (v_sim[-3] / vmax) ** DELTA - (prev_sn / s_sim[-3]) ** 2)
                            a_idm_prev = np.clip(a_idm_prev, -amin, amax)
                            a_error = a_real_prev - a_idm_prev
                            acceleration_errors.append(a_error)
                    else:
                        if i > 0:
                            a_real_prev = real_accel_veh[i-1] if (i-1) < len(real_accel_veh) else 0.0
                            prev_dv = dv_veh[i-1] if (i-1) < len(dv_veh) else 0.0
                            prev_sn = dsafe + v_sim[-2] * tsafe + v_sim[-2] * prev_dv / (2 * np.sqrt(amax * amin))
                            a_idm_prev = amax * (1 - (v_sim[-2] / vmax) ** DELTA - (prev_sn / s_sim[-2]) ** 2)
                            a_idm_prev = np.clip(a_idm_prev, -amin, amax)
                            a_error = a_real_prev - a_idm_prev
                            acceleration_errors.append(a_error)
                        
                        current_dv = dv_veh[i] if i < len(dv_veh) else 0.0
                        sn = dsafe + v_sim[-1] * tsafe + v_sim[-1] * current_dv / (2 * np.sqrt(amax * amin))
                        current_acc_idm = amax * (1 - (v_sim[-1] / vmax) ** DELTA - (sn / s_sim[-1]) ** 2)
                        current_acc_idm = np.clip(current_acc_idm, -amin, amax)
                        
                        v_next_base = v_sim[-1] + current_acc_idm * dt
                        
                        v_ar_correction = 0.0
                        if len(v_history_real) > d:
                            for lag in range(d):
                                if len(v_history_real) > lag + 1:
                                    vt_diff = v_history_real[-1] - v_history_real[-(lag+2)]
                                    if len(idm_acc_history) > lag:
                                        a_lag = idm_acc_history[-(lag+1)]
                                    else:
                                        a_lag = 0.0
                                    
                                    ar_coef = rho_dynamic_val[i, lag]
                                    v_ar_correction += ar_coef * (vt_diff - a_lag * dt)
                        
                        v_next_after_v_correction = v_next_base + v_ar_correction
                        
                        a_ar_correction = 0.0
                        if len(acceleration_errors) >= d:
                            for lag in range(d):
                                if len(acceleration_errors) > lag:
                                    a_ar_correction += rho_dynamic_val[i, lag] * acceleration_errors[-(lag+1)]
                        
                        current_acc_total = current_acc_idm + a_ar_correction
                        current_acc_total = np.clip(current_acc_total, -amin, amax)
                        
                        s_next = s_sim[-1] + (current_dv +0.5*v_ar_correction)* dt+0.5*dt*current_acc_idm * dt
                        s_next = max(1.0, min(200.0, s_next))
                        
                        v_sim.append(v_next_after_v_correction)
                        s_sim.append(s_next)
                        a_sim.append(current_acc_total)
                        
                        if i + 1 < len(vt_veh):
                            v_history_real.append(vt_veh[i + 1])
                        else:
                            v_history_real.append(v_next_after_v_correction)
                        
                        idm_acc_history.append(current_acc_idm)
                
                ar_corrected_speed = np.array(v_sim)
                corrected_spacing = np.array(s_sim)
                corrected_acceleration = np.array(a_sim)
                
                all_speed_predictions.extend(ar_corrected_speed[:n])
                all_spacing_predictions.extend(corrected_spacing[:n])
                all_acceleration_predictions.extend(corrected_acceleration[:n])
                
                sample_vehicle_predictions[veh_id] = {
                    'speed_pred': ar_corrected_speed[:n],
                    'spacing_pred': corrected_spacing[:n],
                    'acceleration_pred': corrected_acceleration[:n],
                    'speed_true': vt_veh,
                    'spacing_true': s_veh,
                    'acceleration_true': real_accel_veh,
                    'ar_coefficients': rho_dynamic_val,
                    'warmup_steps': warmup_steps
                }
                
            else:
                if veh_id in train_data['tracks']:
                    initial_v = train_data['tracks'][veh_id]['last_vt']
                    initial_s = train_data['tracks'][veh_id]['last_s']
                else:
                    initial_v = vt_veh[0] if len(vt_veh) > 0 else 0.0
                    initial_s = s_veh[0] if len(s_veh) > 0 else 10.0
                
                simple_speed = np.full(len(vt_veh), initial_v)
                simple_spacing = np.full(len(s_veh), initial_s)
                simple_acceleration = np.zeros(len(vt_veh))
                
                all_speed_predictions.extend(simple_speed)
                all_spacing_predictions.extend(simple_spacing)
                all_acceleration_predictions.extend(simple_acceleration)
                
                empty_ar_coeffs = np.zeros((len(vt_veh), d))
                all_ar_coefficients_veh.append(empty_ar_coeffs)
                
                sample_vehicle_predictions[veh_id] = {
                    'speed_pred': simple_speed,
                    'spacing_pred': simple_spacing,
                    'acceleration_pred': simple_acceleration,
                    'speed_true': vt_veh,
                    'spacing_true': s_veh,
                    'acceleration_true': real_accel_veh if len(real_accel_veh) == len(simple_speed) else np.zeros(len(simple_speed)),
                    'ar_coefficients': empty_ar_coeffs,
                    'warmup_steps': 0
                }
        
        all_speed_predictions = np.array(all_speed_predictions)
        all_spacing_predictions = np.array(all_spacing_predictions)
        all_acceleration_predictions = np.array(all_acceleration_predictions)
        
        all_samples_speed_predictions.append(all_speed_predictions)
        all_samples_spacing_predictions.append(all_spacing_predictions)
        all_samples_acceleration_predictions.append(all_acceleration_predictions)
        all_samples_ar_coefficients.append(all_ar_coefficients_veh)
        all_samples_vehicle_predictions[sample_idx] = sample_vehicle_predictions
    
    min_len = min(
        len(all_samples_speed_predictions[0]), 
        len(all_samples_spacing_predictions[0]),
        len(all_samples_acceleration_predictions[0]), 
        len(real_acceleration_all),
        len(label_val), 
        len(s_val)
    )
    
    boundary_cut = int(0.05 * min_len)
    valid_indices = slice(boundary_cut, min_len - boundary_cut)
    
    print(f"Excluding boundary segments: using indices {boundary_cut} to {min_len - boundary_cut}")
    
    print("\n" + "="*80)
    print("PER-VEHICLE VALIDATION RESULTS")
    print("="*80)
    
    vehicle_metrics = {}
    
    for veh_id in unique_vehicles:
        veh_speed_preds = []
        veh_spacing_preds = []
        veh_accel_preds = []
        veh_speed_true = None
        veh_spacing_true = None
        veh_accel_true = None
        
        for sample_idx in range(n_samples):
            if veh_id in all_samples_vehicle_predictions[sample_idx]:
                veh_data = all_samples_vehicle_predictions[sample_idx][veh_id]
                veh_speed_preds.append(veh_data['speed_pred'])
                veh_spacing_preds.append(veh_data['spacing_pred'])
                veh_accel_preds.append(veh_data['acceleration_pred'])
                
                if veh_speed_true is None:
                    veh_speed_true = veh_data['speed_true']
                    veh_spacing_true = veh_data['spacing_true']
                    veh_accel_true = veh_data['acceleration_true']
        
        if veh_speed_true is not None and len(veh_speed_true) > 10:
            avg_speed_pred = np.mean(veh_speed_preds, axis=0)
            avg_spacing_pred = np.mean(veh_spacing_preds, axis=0)
            avg_accel_pred = np.mean(veh_accel_preds, axis=0)
            
            veh_boundary_cut = int(0.05 * len(veh_speed_true))
            veh_valid_indices = slice(veh_boundary_cut, len(veh_speed_true) - veh_boundary_cut)
            
            speed_metrics = calculate_metrics(
                veh_speed_true[veh_valid_indices], 
                avg_speed_pred[veh_valid_indices]
            )
            spacing_metrics = calculate_metrics(
                veh_spacing_true[veh_valid_indices], 
                avg_spacing_pred[veh_valid_indices]
            )
            acceleration_metrics = calculate_metrics(
                veh_accel_true[veh_valid_indices],
                avg_accel_pred[veh_valid_indices]
            )
            
            vehicle_metrics[veh_id] = {
                'speed': speed_metrics,
                'spacing': spacing_metrics,
                'acceleration': acceleration_metrics,
                'n_points': len(veh_speed_true[veh_valid_indices])
            }
            
            print(f"\nVehicle {veh_id} (n={vehicle_metrics[veh_id]['n_points']}):")
            print(f"  Speed - RMSE: {speed_metrics['rmse']:.4f} m/s, MAE: {speed_metrics['mae']:.4f} m/s, NRMSE: {speed_metrics['nrmse']:.4f}")
            print(f"  Spacing - RMSE: {spacing_metrics['rmse']:.4f} m, MAE: {spacing_metrics['mae']:.4f} m, NRMSE: {spacing_metrics['nrmse']:.4f}")
            print(f"  Acceleration - RMSE: {acceleration_metrics['rmse']:.4f} m/s², MAE: {acceleration_metrics['mae']:.4f} m/s², NRMSE: {acceleration_metrics['nrmse']:.4f}")
    
    speed_metrics_all = []
    spacing_metrics_all = []
    acceleration_metrics_all = []
    
    for i in range(n_samples): 
        speed_metrics = calculate_metrics(
            label_val[valid_indices], 
            all_samples_speed_predictions[i][valid_indices]
        )
        spacing_metrics = calculate_metrics(
            s_val[valid_indices], 
            all_samples_spacing_predictions[i][valid_indices]
        )
        acceleration_metrics = calculate_metrics(
            real_acceleration_all[valid_indices],
            all_samples_acceleration_predictions[i][valid_indices]
        )
        
        speed_metrics_all.append(speed_metrics)
        spacing_metrics_all.append(spacing_metrics)
        acceleration_metrics_all.append(acceleration_metrics)
    
    avg_speed_metrics = {
        'mse': np.mean([m['mse'] for m in speed_metrics_all]),
        'rmse': np.mean([m['rmse'] for m in speed_metrics_all]),
        'mae': np.mean([m['mae'] for m in speed_metrics_all]),
        'nrmse': np.mean([m['nrmse'] for m in speed_metrics_all])
    }
    
    avg_spacing_metrics = {
        'mse': np.mean([m['mse'] for m in spacing_metrics_all]),
        'rmse': np.mean([m['rmse'] for m in spacing_metrics_all]),
        'mae': np.mean([m['mae'] for m in spacing_metrics_all]),
        'nrmse': np.mean([m['nrmse'] for m in spacing_metrics_all])
    }
    
    avg_acceleration_metrics = {
        'mse': np.mean([m['mse'] for m in acceleration_metrics_all]),
        'rmse': np.mean([m['rmse'] for m in acceleration_metrics_all]),
        'mae': np.mean([m['mae'] for m in acceleration_metrics_all]),
        'nrmse': np.mean([m['nrmse'] for m in acceleration_metrics_all])
    }
    
    print("\n" + "="*80)
    print("VEHICLE PERFORMANCE SUMMARY")
    print("="*80)
    
    if vehicle_metrics:
        speed_rmse_values = [vm['speed']['rmse'] for vm in vehicle_metrics.values()]
        spacing_rmse_values = [vm['spacing']['rmse'] for vm in vehicle_metrics.values()]
        accel_rmse_values = [vm['acceleration']['rmse'] for vm in vehicle_metrics.values()]
        
        print(f"Number of vehicles analyzed: {len(vehicle_metrics)}")
        print(f"\nSpeed RMSE - Mean: {np.mean(speed_rmse_values):.4f}, Std: {np.std(speed_rmse_values):.4f}, "
              f"Min: {np.min(speed_rmse_values):.4f}, Max: {np.max(speed_rmse_values):.4f}")
        print(f"Spacing RMSE - Mean: {np.mean(spacing_rmse_values):.4f}, Std: {np.std(spacing_rmse_values):.4f}, "
              f"Min: {np.min(spacing_rmse_values):.4f}, Max: {np.max(spacing_rmse_values):.4f}")
        print(f"Acceleration RMSE - Mean: {np.mean(accel_rmse_values):.4f}, Std: {np.std(accel_rmse_values):.4f}, "
              f"Min: {np.min(accel_rmse_values):.4f}, Max: {np.max(accel_rmse_values):.4f}")
    
    print("\n" + "="*80)
    print("OVERALL VALIDATION RESULTS")
    print("="*80)
    print(f"Dynamic AR Validation Results (using {n_samples} posterior samples, excluding boundaries):")
    print(f"\nSpeed Metrics:")
    print(f"  RMSE: {avg_speed_metrics['rmse']:.4f} m/s")
    print(f"  NRMSE: {avg_speed_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_speed_metrics['mae']:.4f} m/s")
    
    print(f"\nSpacing Metrics:")
    print(f"  RMSE: {avg_spacing_metrics['rmse']:.4f} m")
    print(f"  NRMSE: {avg_spacing_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_spacing_metrics['mae']:.4f} m")
    
    print(f"\nAcceleration Metrics:")
    print(f"  RMSE: {avg_acceleration_metrics['rmse']:.4f} m/s²")
    print(f"  NRMSE: {avg_acceleration_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_acceleration_metrics['mae']:.4f} m/s²")
    
    return {
        'speed_metrics': avg_speed_metrics,
        'spacing_metrics': avg_spacing_metrics,
        'acceleration_metrics': avg_acceleration_metrics,
        'vehicle_metrics': vehicle_metrics,
        'all_samples_speed_predictions': all_samples_speed_predictions,
        'all_samples_spacing_predictions': all_samples_spacing_predictions,
        'all_samples_acceleration_predictions': all_samples_acceleration_predictions,
        'all_samples_ar_coefficients': all_samples_ar_coefficients,
        'all_samples_vehicle_predictions': all_samples_vehicle_predictions,
        'real_acceleration': real_acceleration_all,
        'individual_params': trace.posterior['parameters'].mean(dim=("chain", "draw")).values,
        'ar_coeffs': trace.posterior['rho_0'].mean(dim=("chain", "draw")).values,
        'valid_indices': valid_indices,
        'n_samples': n_samples,
        'rho_block_info': numerical_block_info,
        'warmup_steps': warmup_steps
    }

def plot_dynamic_ar_validation_results(val_data, validation_results, tau_sec=3):
    vt_val = val_data['vt']
    s_val = val_data['s']
    label_val = val_data['label_v']
    id_idx_val = val_data['id_idx']
    
    all_samples_speed = validation_results['all_samples_speed_predictions']
    all_samples_spacing = validation_results['all_samples_spacing_predictions']
    all_samples_acceleration = validation_results['all_samples_acceleration_predictions']
    all_samples_ar_coefficients = validation_results['all_samples_ar_coefficients']
    real_acceleration = validation_results['real_acceleration']
    valid_indices = validation_results['valid_indices']
    n_samples = validation_results['n_samples']
    
    unique_vehicles = np.unique(id_idx_val)
    n_vehicles = len(unique_vehicles)
    
    dt = 0.5
    tau = int(tau_sec / dt)
    
    title_fontsize = 14
    label_fontsize = 12
    legend_fontsize = 11
    tick_fontsize = 10
    
    for veh_idx, veh_id in enumerate(unique_vehicles):
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        
        mask = (id_idx_val == veh_id)
        if np.sum(mask) > 0:
            veh_data_points = min(np.sum(mask), len(all_samples_speed[0]))
            time_points = np.arange(veh_data_points) * dt
            
            axes[0, 0].plot(time_points, label_val[mask][:veh_data_points], 'k-', 
                           linewidth=3, alpha=0.9)
            
            veh_speed_samples = np.array([sample[mask][:veh_data_points] for sample in all_samples_speed])
            
            lower_5 = np.percentile(veh_speed_samples, 5, axis=0)
            upper_95 = np.percentile(veh_speed_samples, 95, axis=0)
            lower_25 = np.percentile(veh_speed_samples, 25, axis=0)
            upper_75 = np.percentile(veh_speed_samples, 75, axis=0)
            
            axes[0, 0].fill_between(time_points, lower_5, upper_95, 
                                   alpha=0.3, color='red')
            axes[0, 0].fill_between(time_points, lower_25, upper_75, 
                                   alpha=0.5, color='red', label=f'Driver {veh_id+1}')
            
            mean_speed = np.mean(veh_speed_samples, axis=0)
            axes[0, 0].plot(time_points, mean_speed, 'b-', 
                           linewidth=2, alpha=0.8)
            
            axes[0, 0].set_xlabel('Time (s)', fontsize=label_fontsize)
            axes[0, 0].set_ylabel('Speed (m/s)', fontsize=label_fontsize)
            axes[0, 0].legend(fontsize=legend_fontsize)
            axes[0, 0].grid(True, alpha=0.3)
            axes[0, 0].tick_params(axis='both', which='major', labelsize=tick_fontsize)
            
            axes[0, 1].plot(time_points, s_val[mask][:veh_data_points], 'k-', 
                           linewidth=3, alpha=0.9)
            
            veh_spacing_samples = np.array([sample[mask][:veh_data_points] for sample in all_samples_spacing])
            
            lower_5_s = np.percentile(veh_spacing_samples, 5, axis=0)
            upper_95_s = np.percentile(veh_spacing_samples, 95, axis=0)
            
            axes[0, 1].fill_between(time_points, lower_5_s, upper_95_s, 
                                   alpha=0.3, color='green', label=f'Driver {veh_id+1}')
            
            mean_spacing = np.mean(veh_spacing_samples, axis=0)
            axes[0, 1].plot(time_points, mean_spacing, 'g-', 
                           linewidth=2, alpha=0.8)
            
            axes[0, 1].set_xlabel('Time (s)', fontsize=label_fontsize)
            axes[0, 1].set_ylabel('Spacing (m)', fontsize=label_fontsize)
            axes[0, 1].legend(fontsize=legend_fontsize)
            axes[0, 1].grid(True, alpha=0.3)
            axes[0, 1].tick_params(axis='both', which='major', labelsize=tick_fontsize)
            
            axes[0, 2].plot(time_points, real_acceleration[mask][:veh_data_points], 'k-', 
                           linewidth=3, alpha=0.9)
            
            veh_accel_samples = np.array([sample[mask][:veh_data_points] for sample in all_samples_acceleration])
            
            lower_5_a = np.percentile(veh_accel_samples, 5, axis=0)
            upper_95_a = np.percentile(veh_accel_samples, 95, axis=0)
            
            axes[0, 2].fill_between(time_points, lower_5_a, upper_95_a, 
                                   alpha=0.3, color='orange', label=f'Driver {veh_id+1}')
            
            mean_acceleration = np.mean(veh_accel_samples, axis=0)
            axes[0, 2].plot(time_points, mean_acceleration, 'c-', 
                           linewidth=2, alpha=0.8)
            
            axes[0, 2].set_xlabel('Time (s)', fontsize=label_fontsize)
            axes[0, 2].set_ylabel('Acceleration (m/s²)', fontsize=label_fontsize)
            axes[0, 2].legend(fontsize=legend_fontsize)
            axes[0, 2].grid(True, alpha=0.3)
            axes[0, 2].tick_params(axis='both', which='major', labelsize=tick_fontsize)
            
            if len(all_samples_ar_coefficients) > 0 and veh_idx < len(all_samples_ar_coefficients[0]):
                sample_ar_coeffs = all_samples_ar_coefficients[0][veh_idx]
                if len(sample_ar_coeffs) >= veh_data_points:
                    for lag in range(min(4, sample_ar_coeffs.shape[1])):
                        axes[1, 0].plot(time_points, sample_ar_coeffs[:veh_data_points, lag], 
                                       alpha=0.8)
                
                axes[1, 0].plot([], [], color='blue', label=f'Driver {veh_id+1}')
                
                axes[1, 0].set_xlabel('Time (s)', fontsize=label_fontsize)
                axes[1, 0].set_ylabel('AR Coefficient Value', fontsize=label_fontsize)
                axes[1, 0].legend(fontsize=legend_fontsize)
                axes[1, 0].grid(True, alpha=0.3)
                axes[1, 0].tick_params(axis='both', which='major', labelsize=tick_fontsize)
                
                for block_idx in range(0, veh_data_points, tau):
                    if block_idx < veh_data_points:
                        axes[1, 0].axvline(x=block_idx*dt, color='gray', linestyle='--', alpha=0.5)
            
            if len(all_samples_ar_coefficients) > 0 and veh_idx < len(all_samples_ar_coefficients[0]):
                all_veh_ar_samples = []
                for sample_idx in range(min(10, len(all_samples_ar_coefficients))):
                    if veh_idx < len(all_samples_ar_coefficients[sample_idx]):
                        ar_coeffs = all_samples_ar_coefficients[sample_idx][veh_idx]
                        if len(ar_coeffs) >= veh_data_points:
                            all_veh_ar_samples.append(ar_coeffs[:veh_data_points, 0])
                
                if len(all_veh_ar_samples) > 0:
                    all_veh_ar_samples = np.array(all_veh_ar_samples)
                    lower_5_ar = np.percentile(all_veh_ar_samples, 5, axis=0)
                    upper_95_ar = np.percentile(all_veh_ar_samples, 95, axis=0)
                    mean_ar = np.mean(all_veh_ar_samples, axis=0)
                    
                    axes[1, 1].fill_between(time_points, lower_5_ar, upper_95_ar, 
                                          alpha=0.3, color='purple', label=f'Driver {veh_id+1}')
                    axes[1, 1].plot(time_points, mean_ar, 'purple', 
                                   linewidth=2)
                    
                    axes[1, 1].set_xlabel('Time (s)', fontsize=label_fontsize)
                    axes[1, 1].set_ylabel('AR Coefficient Value', fontsize=label_fontsize)
                    axes[1, 1].legend(fontsize=legend_fontsize)
                    axes[1, 1].grid(True, alpha=0.3)
                    axes[1, 1].tick_params(axis='both', which='major', labelsize=tick_fontsize)
            
            speed_errors = label_val[mask][:veh_data_points] - mean_speed
            axes[1, 2].plot(time_points, speed_errors, 'r-', alpha=0.7, label=f'Driver {veh_id+1}')
            axes[1, 2].axhline(y=0, color='k', linestyle='-', alpha=0.5)
            axes[1, 2].fill_between(time_points, speed_errors, 0, alpha=0.3, color='red')
            
            axes[1, 2].set_xlabel('Time (s)', fontsize=label_fontsize)
            axes[1, 2].set_ylabel('Error (m/s)', fontsize=label_fontsize)
            axes[1, 2].legend(fontsize=legend_fontsize)
            axes[1, 2].grid(True, alpha=0.3)
            axes[1, 2].tick_params(axis='both', which='major', labelsize=tick_fontsize)
        
        fig.suptitle(f'Dynamic AR Model Validation - Driver {veh_id+1}', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

In [ ]:
calibration_results = run_dynamic_ar_calibration_only(
     ar_idm_data, 
     ar_order=1, 
     tau_sec=5
 )

In [ ]:

validation_results = run_dynamic_ar_validation_only(
     calibration_results, 
    n_posterior_samples=100,
     ar_order=1,
     tau_sec=5
 )